In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

In [27]:
df = pd.read_csv("authorlist.csv")

n = 3 #Edit this
url = df['link'][n]
author = df['author'][n]
save_name = author.split('-')[0].replace(' ','')+'.csv'

print(save_name)
print(link)

ChếLanViên.csv
https://www.thivien.net/Ch%C3%A2u-La-Vi%E1%BB%87t/T%C3%A2m-s%E1%BB%B1-v%E1%BB%9Bi-s%C3%B4ng-B%E1%BB%93/poem-D2UjhNsD0O5Z7OgDyg8U6Q


,Unnamed: 0,author,link
0,0,Châu La Việt - Lê Khánh Hoài,https://www.thivien.net/Ch%C3%A2u-La-Vi%E1%BB%...
1,1,Châu Nho - Nguyễn Minh Châu,https://www.thivien.net/Ch%C3%A2u-Nho/author-c...
2,2,Châu Thượng Vân,https://www.thivien.net/Ch%C3%A2u-Th%C6%B0%E1%...
3,3,Chế Lan Viên - Phan Ngọc Hoan,https://www.thivien.net/Ch%E1%BA%BF-Lan-Vi%C3%...
4,4,Chi Mai,https://www.thivien.net/Chi-Mai/author-V6sWiff...
5,5,Chim Trắng - Hồ Văn Ba,https://www.thivien.net/Chim-Tr%E1%BA%AFng/aut...
6,6,Chính Hữu - Trần Đình Đắc,https://www.thivien.net/Ch%C3%ADnh-H%E1%BB%AFu...
7,7,Chu Đường Anh - 周堂瑛,https://www.thivien.net/Chu-%C4%90%C6%B0%E1%BB...
8,8,Chu Hoạch,https://www.thivien.net/Chu-Ho%E1%BA%A1ch/auth...
9,9,Chu Huân,https://www.thivien.net/Chu-Hu%C3%A2n/author-P...


In [12]:
def find_links(url):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    web_byte = urlopen(req).read()
    html = web_byte.decode('utf-8')
    soup = BeautifulSoup(html, 'lxml')

    link_list = set()
    for link in soup.find_all("a"):
        if '/poem' in str(link):
            link = 'https://www.thivien.net'+link['href']
            link_list.add(link)
    
    return list(link_list)

In [28]:
link_list = find_links(url)
len(link_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



490

In [14]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    raw_html = re.sub('<br/>', '@', raw_html)
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def split_lines(url):
    lines = []
    
    #Web scraping 
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    web_byte = urlopen(req).read()
    html = web_byte.decode('utf-8')
    
    #Find poem content
    soup = BeautifulSoup(html, 'lxml')
    poem = soup.findAll("div", {"class": "poem-content"})
    
    #Split lines
    if poem:
        for p in poem:
            p = cleanhtml(str(p))
            p = re.sub('(\\\w+)|' '|(\*)+','',str(p))
            p = re.sub('</span><i class="fa fa-info-circle popupcmt">','',str(p))
            lines = re.split('@',str(p))[1:-1]
            lines = list(filter(None, lines))
    return lines

In [15]:
rawdata = []

for link in link_list:
    rawdata.append(split_lines(link))

len(rawdata)

2

In [16]:
import itertools
rawdata = list(filter(None,rawdata)) #Filter empty str from request limits
flat_rawdata = list(itertools.chain(*rawdata)) #Flatten list
len(flat_rawdata)

45

In [17]:
import csv
with open(save_name,'w', encoding='UTF-8', newline='') as resultFile:
    wr = csv.writer(resultFile, dialect='excel')
    wr.writerow(flat_rawdata)